Import required libraries:

In [143]:
import requests
import json
import os
import pandas as pd

### Set SCRAPE=True in order to download the data from FB

In [144]:
SCRAPE = False

Define a function to return an empty list if string is not JSON formatted:

In [145]:
def read_json(path):
    try:
        return json.loads(open(path, "r").read())
    except json.decoder.JSONDecodeError:
        return []

Try to read "posts.json". If it does not exist, create it.

In [146]:
try:
    posts = read_json("posts.json")
except FileNotFoundError:
    f = open("posts.json", "w")
    f.close
    posts = []

Create a supplemental function to simplify API 

In [147]:
def get(url_):
    return json.loads(requests.get(url_).text)

def post_exists(post_id):
    return len(list(filter(lambda x: x['id'] == post_id, posts))) > 0

Define gruop ID variables:

In [148]:
TECHNION_CONFESSIONS_ID = "134517547222780"
TAU_CONFESSIONS_ID = "561380070875128"
IDC_CONFESSIONS_ID = "199527394120566"

Authenticate to Graph API:

In [149]:
# TOKEN = "EAACEdEose0cBAEIYEOmJ51kfOHPinwL2ychqVpFT0D9ezMjfe1pMKyH0lP2ZA9RuZBH3wQeJ0EfEboHEqDqy7Gq8PSjIcHgNv1lRyC3ZBMk0CdbdzWckHsqNTRatwPZBvkMBfic2P1jwg5vZAZCHKbHdvFfMjjsHGjeVsY7AqEbpWiUBEb0d1zsKJgJwQw3wTWs6ZBb4WSOOQZDZD"

s = "6cc937f2a9dbc9df92600f365c777d1a"
i = "652869818252649"
u = "https://graph.facebook.com/oauth/access_token?client_id={id}&client_secret={secret}&grant_type=client_credentials"

TOKEN = get(u.format(id=i, secret=s))['access_token']

Setup Graph API host and API node:

In [150]:
HOST = "https://graph.facebook.com/"
API_NODE = "v2.12/"

Declare custom node - in this case, the 'Technion Confessions' group feed:

In [151]:
TECHNION_CONFESSIONS_NODE = "{}/feed".format(TECHNION_CONFESSIONS_ID)
TAU_CONFESSIONS_NODE = "{}/feed".format(TAU_CONFESSIONS_ID)
IDC_CONFESSIONS_NODE = "{}/feed".format(IDC_CONFESSIONS_ID)

PAGES = [
    (TECHNION_CONFESSIONS_NODE, "TECHNION"),
    (TAU_CONFESSIONS_NODE, "TAU"),
    (IDC_CONFESSIONS_NODE, "IDC"),
]

Build the requested URL - should be the same for the most of the calls:

In [152]:
def build_url(node):
    return HOST + API_NODE + node + "?access_token={}".format(TOKEN)

## ! Edit the algorithm to add new posts - aka continue from the last post on

The main post scraping function:

In [153]:
def get_posts(url, origin):
    res = get(url)
    if 'data' in res:
        for post in res['data']:
            if not post_exists(post['id']):
                post['origin'] = origin
                posts.append(post)
    if 'paging' in res:
        if 'next' in res['paging']:
            return get_posts(res['paging']['next'], origin)

If SCRAPE, the script will fill "posts.json" with the posts:

In [154]:
if SCRAPE:
    for node, name in PAGES:
        get_posts(build_url(node), name)
    with open("posts.json", "w") as f:
        f.write(json.dumps(posts))

Show the head of the posts table:

In [155]:
df = pd.read_json(open("posts.json", "r").read())
df.head()

,created_time,id,message,origin,story
0,2018-02-12 08:15:17,1.345175e+29,#2273 הדבר שהכי מלחיץ אותי הוא הסמינר שאקח בסמ...,TECHNION,NaN
1,2018-02-12 08:14:31,1.345175e+29,"#2272 סמסטר חורף קודם, ירדתי לכיוון נשר מאולמן...",TECHNION,NaN
2,2018-02-12 08:12:20,1.345175e+29,"#2271 בדרך כלל אני בן אדם מאוד שמח, או שזה מה ...",TECHNION,NaN
3,2018-02-12 08:08:57,1.345175e+29,#2270 אני מוכן להיות אבא כבר מגיל 14. תמיד חלמ...,TECHNION,NaN
4,2018-02-12 08:07:29,1.345175e+29,#2269 אני בת 26 והדייט המושלם לדעתי הוא פשוט ל...,TECHNION,NaN


In [166]:
df["id"] = df["message"].str.findall("(#+\d+){1}").str.get(0)
df["reply_to"] = df["id"] = df["message"].str.findall("(#+\d+){1}").str.get(1)
df

,created_time,id,message,origin,story,reply_to
0,2018-02-12 08:15:17,NaN,#2273 הדבר שהכי מלחיץ אותי הוא הסמינר שאקח בסמ...,TECHNION,NaN,NaN
1,2018-02-12 08:14:31,NaN,"#2272 סמסטר חורף קודם, ירדתי לכיוון נשר מאולמן...",TECHNION,NaN,NaN
2,2018-02-12 08:12:20,NaN,"#2271 בדרך כלל אני בן אדם מאוד שמח, או שזה מה ...",TECHNION,NaN,NaN
3,2018-02-12 08:08:57,NaN,#2270 אני מוכן להיות אבא כבר מגיל 14. תמיד חלמ...,TECHNION,NaN,NaN
4,2018-02-12 08:07:29,NaN,#2269 אני בת 26 והדייט המושלם לדעתי הוא פשוט ל...,TECHNION,NaN,NaN
5,2018-02-12 08:04:56,NaN,"#2268 יש לי חבר שהוא איטי יותר בהבנה, הבעיה הי...",TECHNION,NaN,NaN
6,2018-02-12 08:03:34,NaN,#2267 לפני כמה ימים יצאתי עם בחורה שמוצאת חן ב...,TECHNION,NaN,NaN
7,2018-02-12 08:01:57,NaN,#2266 באסה שאני לא יכולה להגיע לטכניון בפיג׳מה...,TECHNION,NaN,NaN
8,2018-02-12 08:00:45,NaN,#2265 אני קנאית בטירוף וזה משגע אותי. לחבר שלי...,TECHNION,NaN,NaN
9,2018-02-12 07:59:23,NaN,"#2264 במקום ללמוד נכנסתי לכתבות על ""איך לענג א...",TECHNION,NaN,NaN


Get the IDs

Get ID of the posts that the post is replying to

Get the posts likes

Get the comments of the posts

Get the likes for each comment

Get the comments for each comment

Get the name of the commenter (for each comment or reply)

Get the commenter gender